In [1]:
import os
workDir = os.getcwd()
user = 'mkilpatr'
print(workDir)

/Users/Matthew/Documents/SchoolWork/physics/GradNotes/ParticlePhysics/PostdocResearch/HGC/EstToolsSUSY/HGCStudies


In [2]:
%run GetExcel.ipynb

sensorfile = workDir + '/Fiducials/sensorfiducialtrialrun.xls'
pcbfile = workDir + '/Fiducials/pcbfiducialtrialrun.xls'
fiducialfile = workDir + '/Fiducials/GantryTrayFiducialData.xls'

sensor = GetExcel(sensorfile, True, [])
sensor.read()

fiducial = GetExcel(fiducialfile, True, [])
fiducial.read()

pcb = GetExcel(pcbfile, True, [])
pcb.read()

FILEPATHS MAY BE WRONG!! CANNOT FIND /Users/Matthew/Documents/SchoolWork/physics/GradNotes/ParticlePhysics/PostdocResearch/HGC/EstToolsSUSY/HGCStudies/Fiducials/pcbfiducialtrialrun.xls


In [3]:
%run ReadDataframe.ipynb

if sensor.exists:
    senDF = ReadDataframe(sensor.df)
    senDF.readComponentFiducials()

if fiducial.exists:
    fidDF = ReadDataframe(fiducial.df)
    fidDF.readTrayFiducials()

if pcb.exists:
    pcbDF = ReadDataframe(pcb.df)
    senDF.readComponentFiducials()

In [7]:
%run QualityControl.ipynb

if len(senDF.pointsP1):
    sensorP1 = QualityControl(fidDF.trayPoints, fidDF.P1, senDF.pointsP1, 'P1')
    sensorP1.RunTransform()
    sensorP1.Rating()

if len(senDF.pointsP2):
    sensorP2 = QualityControl(fidDF.trayPoints, fidDF.P2, senDF.pointsP2, 'P2')
    sensorP2.RunTransform()
    sensorP2.Rating()

if pcb.exists:
    if len(pcbDF.pointsP1):
        pcbP1 = QualityControl(fidDF.trayPoints, fidDF.P1, pcbDF.pointsP1, 'P1')
        pcbP1.RunTransform()
        pcbP1.Rating()

    if len(pcbDF.pointsP2):
        pcbP2 = QualityControl(fidDF.trayPoints, fidDF.P2, pcbDF.pointsP2, 'P2')
        pcbP2.RunTransform()
        pcbP2.Rating()

The angle of the sensor is BAD in position P1
The placement of the sensor is BAD in position P1
